### Develop Python code that replicates the calculations in Appendix B of Messié et al. 2019

The paper is at https://doi.org/10.1016/j.pocean.2018.12.010 and the Matlab code is at https://bitbucket.org/messiem/toolbox_blprocess/src/master/bl_window_smoothing.m

In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join("../src/data"))
if module_path not in sys.path:
    sys.path.append(module_path)
import xarray as xr
import holoviews as hv
import hvplot.pandas
import ipywidgets as widgets
from logs2netcdfs import BASE_PATH, MISSIONNETCDFS

# Assumes that data have been processed locally using src/data/process_dorado.py
# Pick a typical Diamonnd mission with lots of peak flashes
# Share this view URL for this mission: http://stoqs.mbari.org/p/w2UREyQ
auv_name = "dorado"
mission = "2021.102.02"

In [ ]:
netcdfs_dir = os.path.join(BASE_PATH, auv_name, MISSIONNETCDFS, mission)
opendap_base = os.path.join("http://dods.mbari.org:8181/opendap/data/auvctd/surveys/", mission.split('.')[0], 'netcdf')
# Use `opendap_base` to test new hyrax-docker opendap server
ds_align = xr.open_dataset(os.path.join(netcdfs_dir, f"{auv_name}_{mission}_align.nc"))
#ds_1S = xr.open_dataset(os.path.join(opendap_base, f"{auv_name}_{mission}_1S.nc"))
print("New hyrax-docker link: " + os.path.join(opendap_base, f"{auv_name}_{mission}_align.nc"))
ds_align

In [ ]:
# Focus on a 10 minute period that is here in STOQS: http://stoqs.mbari.org/p/V3F_1d0
# Do not commit to the repo the output of this cell, it is too large - about 70 MB
hv.extension("bokeh")   # See https://github.com/holoviz/holoviews/issues/4861#issuecomment-1239739743
raw_plot_2m = ds_align["biolume_raw"].sel(biolume_time60hz=slice("2021-04-13 10:00:00", "2021-04-13 10:10:00")).to_pandas().rename("raw").hvplot()
raw_plot_2m